# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
# coding=utf-8
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
import PIL.Image
import random
#config = tf.ConfigProto() 
#config.gpu_options.allow_growth = True 
#sess = tf.Session(config=config)


from collections import defaultdict
from io import StringIO
# from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from matplotlib import pyplot as plt




## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util



# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_NAME = 'object_detection/faster_rcnn_resnet101_coco_2018_01_28'
MODEL_NAME = 'object_detection/faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# PATH_TO_FROZEN_GRAPH = 'C:\\Users\\Administrator\\Desktop\\models\\research\\object_detection\\faster_rcnn_inception_v2_coco_2017_11_08\\frozen_inference_graph.pb'
PATH_TO_FROZEN_GRAPH = 'G:\\zzx\models\\research\\object_detection\\faster_rcnn_inception_v2_coco_2017_11_08\\frozen_inference_graph.pb'


# List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = 'C:\\Users\\Administrator\\Desktop\\models\\research\\object_detection\\data\\mscoco_label_map.pbtxt'
PATH_TO_LABELS = 'G:\\zzx\\models\\research\\object_detection\\data\\mscoco_label_map.pbtxt'

NUM_CLASSES = 90

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
# detection_graph = tf.Graph()
# with detection_graph.as_default():
#   od_graph_def = tf.GraphDef()
#   with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
#     serialized_graph = fid.read()
#     od_graph_def.ParseFromString(serialized_graph)
#     tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
#category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

AttributeError: module 'tensorflow' has no attribute 'gfile'

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
import glob
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = "C:\\Users\\Administrator\\Desktop\\models\\picture\\objectdetect\\adv-*"
PATH_TO_TEST_IMAGES_DIR = "G:\\zzx\\models\\picture\\objectdetect\\adv-*"
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 2) ]
TEST_IMAGE_PATHS = glob.glob(PATH_TO_TEST_IMAGES_DIR)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

print(TEST_IMAGE_PATHS)

from tensorflow.python import pywrap_tensorflow

checkpoint_path=os.path.join(PATH_TO_CKPT)
reader=pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map=reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print('tensor_name:' ,key)

In [ ]:

image_path="G:\\zzx\\models\\research\\object_detection\\test_images\\1.jpg"
image = Image.open(image_path).convert("RGB")
image=image.resize((1920,1080))


# # the array based representation of the image will be used later in order to prepare the
# # result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image.copy())

In [ ]:
#%%
#定义类adv_model 封装生成对抗样本的功能
class adv_model():
    def __init__(self):
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        self.tensor_dict = {}
        self.patch_shape=(1, 1080, 1920, 3)
        self._make_model_and_ops()
            

    def _make_model_and_ops(self):
        
        with self.sess.graph.as_default():
            
            # Tensors are post-fixed with an underscore!
            self.image_input_ = tf.placeholder(tf.float32, shape=self.patch_shape, name='image_input_placeholder')
            self.patch_ = tf.get_variable('patch', self.patch_shape, dtype=tf.float32, initializer=tf.zeros_initializer)
            self.patch_placeholder_ = tf.placeholder(dtype=tf.float32, shape=self.patch_shape, name='patch_placeholder')
            self.assign_patch_ = tf.assign(self.patch_, self.patch_placeholder_)
                
            #self.clipped_patch_ = tf.tanh(patch_)
            patched_input_ = self.patch_+self.image_input_/255.0
            #patched_input_ = self.clipped_patch_+self.image_input_placeholder
            #高级阶段函数 可以结算梯度
            #This operation has a gradient and thus allows for training min and max values.
            #https://tensorflow.google.cn/api_docs/python/tf/quantization/fake_quant_with_min_max_vars
            self.patched_input_ = tf.fake_quant_with_min_max_vars(patched_input_*255.0, min=0, max=255)
            
            # Create placeholders for NMS RPN inputs
            self.rpn_nms_bboxes_placeholder_ = tf.placeholder(tf.float32, shape=(None, 4), name='rpn_nms_bboxes')
            self.rpn_nms_indices_placeholder_ = tf.placeholder(tf.int32, shape=(None), name='rpn_nms_indices')
            
            
            #image_tensor:0到输出之间存在无法计算梯度的op，因此需要直接操作Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0
            def create_graph(dirname):
                with tf.gfile.GFile(dirname, 'rb') as f:
                    graph_def = self.sess.graph_def
                    graph_def.ParseFromString(f.read())

                    _ = tf.import_graph_def(graph_def, name='adv_model',
                        input_map={
                           'Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0':self.patched_input_,
                           'Reshape_7:0':self.rpn_nms_bboxes_placeholder_,
                           'Reshape_8:0':self.rpn_nms_indices_placeholder_
                            }
                                           )

            create_graph(PATH_TO_FROZEN_GRAPH) 
            
            # Recreate tensors we just replaced in the input_map
            self.rpn_nms_bboxes_ = tf.reshape( self.graph.get_tensor_by_name('adv_model/Reshape_6:0'), 
                                              self.graph.get_tensor_by_name('adv_model/stack_3:0'), 
                                              name='adv_model/Reshape_7')
            self.rpn_nms_indices_ = tf.reshape(self.graph.get_tensor_by_name('adv_model/mul_1:0'), 
                                               self.graph.get_tensor_by_name('adv_model/Reshape_8/shape:0'),
                                               name='adv_model/Reshape_8') 

            # Second-stage Class Loss
            self.second_stage_cls_scores_ = self.graph.get_tensor_by_name('adv_model/SecondStagePostprocessor/convert_scores:0')
            second_stage_cls_logits_ = self.graph.get_tensor_by_name('adv_model/SecondStagePostprocessor/scale_logits:0')
            self.second_stage_cls_labels_ = tf.placeholder(tf.float32, shape=second_stage_cls_logits_.shape, name='second_stage_cls_labels')
            second_stage_cls_losses_ = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.reshape(self.second_stage_cls_labels_, (-1, self.second_stage_cls_labels_.shape[2])),
                                                                                      logits=tf.reshape(second_stage_cls_logits_, (-1, second_stage_cls_logits_.shape[2]))) 
            second_stage_cls_losses_ = tf.reshape(second_stage_cls_losses_, (-1, self.second_stage_cls_labels_.shape[1]))
            second_stage_cls_losses_ = tf.divide(second_stage_cls_losses_, tf.to_float(self.second_stage_cls_labels_.shape[1]))
            self.second_stage_cls_loss_ = tf.reduce_sum(second_stage_cls_losses_)
            
            
            # Second-stage bounding boxes
            self.second_stage_loc_bboxes_ = self.graph.get_tensor_by_name('adv_model/SecondStagePostprocessor/Reshape_4:0')
           
            # Sum of weighted losses
            self.loss_ = self.second_stage_cls_loss_

            # Train our attack by only training on the patch variable
            #self.train_op_ = tf.train.GradientDescentOptimizer(0.1).minimize(self.loss_, var_list=[patch_])
            self.train_op_ = tf.train.AdamOptimizer(0.01).minimize(self.loss_, var_list=[self.patch_])
            
            # 初始化参数  非常重要 Adam的参数也需要这样初始化 GradientDescent可以省略这一步
            self.sess.run(tf.global_variables_initializer())
                
            #初始化
            self.sess.run(self.assign_patch_, {self.patch_placeholder_: np.zeros(self.patch_shape)})

    def inference_rpn(self, images):
        feed_dict = { self.image_input_: images }
        
        tensors = [self.rpn_nms_bboxes_,
                   self.rpn_nms_indices_ ]

        rpn_nms_bboxes, rpn_nms_indices = self.sess.run(tensors, feed_dict)
        
        return rpn_nms_bboxes, rpn_nms_indices

    #预测并绘图
    def inference_draw(self, images):

        patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes = self.inference(images)
        patched_imgs = patched_imgs.astype(np.uint8)
        
        
        scores = second_stage_cls_scores[0]
        bboxes = second_stage_loc_bboxes[0]
        
        sorted_classes = np.argsort(scores[:, 1:], axis=1)
        sorted_scores = scores[:, 1:].copy()
        sorted_bboxes = bboxes.copy()
        
        
#         print("sorted_classes")
#         print(sorted_classes.shape)
#         print(sorted_classes)

        for i, ordering in enumerate(sorted_classes):
            sorted_scores[i, :] = scores[i, ordering+1]
            sorted_bboxes[i, :] = bboxes[i, ordering, :]

        sorted_classes += 1

        
        
        # Visualization of the results of a detection.
        show_image_np=vis_util.visualize_boxes_and_labels_on_image_array(
              patched_imgs[0].copy(),
              sorted_bboxes[:, -1, :],
              sorted_classes[:, -1].astype(np.int32),
              sorted_scores[:, -1],
              category_index,
              use_normalized_coordinates=False,
              max_boxes_to_draw=5,
              min_score_thresh=0.2,
              line_thickness=2,
              skip_scores=True,
              skip_labels=False)


        plt.figure(figsize=(12,8))
        plt.imshow(show_image_np)
#         PIL.Image.fromarray(patched_imgs[0].copy()).save('perturbation.jpg')
 
        return None
        

    def inference(self, images):
        rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images)

        feed_dict = { self.image_input_: images,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices }

        tensors = [ self.patched_input_,
                    self.second_stage_cls_scores_,
                    self.second_stage_loc_bboxes_ ]

        patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes = self.sess.run(tensors, feed_dict)
        patched_imgs = patched_imgs.astype(np.uint8)
        
        return patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes   

    
    
    def train_step(self, images, second_stage_cls_labels):
        
        rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images)
        
        feed_dict = { 
                      self.image_input_: images,
                      self.second_stage_cls_labels_: second_stage_cls_labels,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices, }
        
        
        tensors = [ self.train_op_,
                    self.loss_]

        train_op, loss= self.sess.run(tensors, feed_dict)

        return loss
    
    def get_patch(self):
        
        patch=self.sess.run(self.patch_)
        patch = np.round(patch[0]*255.0).astype(np.uint8)
        
        return patch
#%%


In [ ]:
adv_model_instance = adv_model()
# image_path="G:\\zzx\\models\\research\\object_detection\\test_images\\4.jpg"
# image = Image.open(image_path).convert("RGB")
# image=image.resize((1920,1080))


# # # the array based representation of the image will be used later in order to prepare the
# # # result image with boxes and labels on it.
# image_np = load_image_into_numpy_array(image.copy())


#从person 1 变为horse 19 
TARGET_CLASS = 19
FROM_CLASS = 3 

def create_target_labels(scores, from_class, to_class):
    target_labels = np.zeros_like(scores)
    classes = np.argmax(scores[:, :, 1:], axis=2)+1
    #print(classes)
    
    for i, _ in enumerate(classes):
        for j, cls in enumerate(classes[i]):
            cls = to_class # Just perturb all of them!
            target_labels[i, j, cls] = 1 # set to 1 for targeted attack

    return target_labels


_, scores, _  = adv_model_instance.inference(np.expand_dims(image_np.copy(),0))
#print(scores)
#print(scores.shape)

target_labels = create_target_labels(scores, FROM_CLASS, TARGET_CLASS)
#print(target_labels.shape)


In [ ]:
#开始迭代求解 
#@param ["1: person", "2: bicycle", "3: car", "4: motorcycle", "5: airplane", "6: bus", "7: train", "8: truck", "9: boat", "10: traffic light", "11: fire hydrant", "13: stop sign", "14: parking meter", "15: bench", "16: bird", "17: cat", "18: dog", "19: horse", "20: sheep", "21: cow", "22: elephant", "23: bear", "24: zebra", "25: giraffe", "27: backpack", "28: umbrella", "31: handbag", "32: tie", "33: suitcase", "34: frisbee", "35: skis", "36: snowboard", "37: sports ball", "38: kite", "39: baseball bat", "40: baseball glove", "41: skateboard", "42: surfboard", "43: tennis racket", "44: bottle", "46: wine glass", "47: cup", "48: fork", "49: knife", "50: spoon", "51: bowl", "52: banana", "53: apple", "54: sandwich", "55: orange", "56: broccoli", "57: carrot", "58: hot dog", "59: pizza", "60: donut", "61: cake", "62: chair", "63: couch", "64: potted plant", "65: bed", "67: dining table", "70: toilet", "72: tv", "73: laptop", "74: mouse", "75: remote", "76: keyboard", "77: cell phone", "78: microwave", "79: oven", "80: toaster", "81: sink", "82: refrigerator", "84: book", "85: clock", "86: vase", "87: scissors", "88: teddy bear", "89: hair drier", "90: toothbrush"]
print(image_path)
epochs=3

for epoch in range(epochs):
    loss = adv_model_instance.train_step(np.expand_dims(image_np.copy(),0),target_labels)

    
    #显示中间结果
    if (epoch % 50) == 2:  
#         print("epoch={} loss={}".format(epoch,loss))
        adv_model_instance.inference_draw(np.expand_dims(image_np.copy(),0))

img = image_np.astype(np.int16)#此步是为了避免像素点小于0，大于255的情况
mu =0
sigma = 20
for i in range(img.shape[0]):
     for j in range(img.shape[1]):
        for k in range(img.shape[2]):
            img[i,j,k] = img[i,j,k] +random.gauss(mu=mu,sigma=sigma)
img[img>255] = 255
img[img<0] = 0
img = img.astype(np.uint8)
adv_model_instance.inference_draw(np.expand_dims(img,0))


# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.

# inference_model_instance.inference_draw(np.expand_dims(img,0))
